# Generate simulated data to link

In this case study, we imagine running PVS on the 2030 Census Unedited File (CUF) -- see the main notebook for more details, including references used throughout this notebook.
This notebook creates input (CUF) and reference files approximating what would be used in such a PVS process.

In [1]:
import pseudopeople as psp
import pandas as pd, numpy as np

In [2]:
psp.__version__

'0.7.1'

## Load simulated data

### Noise configuration

In order to give ourselves more of a challenge, we significantly increase the amount of noise
from the pseudopeople defaults.

In [3]:
default_configuration = psp.get_config()

In [4]:
# Helper functions for changing the default configuration according to a pattern
def column_noise_value(dataset, column, noise_type, default_value):
    if dataset in ('decennial_census', 'taxes_w2_and_1099', 'social_security'):
        if noise_type == "make_typos":
            if column == "middle_initial":
                # 5% of middle initials (which are all a single token anyway) are wrong.
                return {"cell_probability": 0.05, "token_probability": 1}
            elif column in ("first_name", "last_name", "street_name"):
                # 10% of these text columns were entered carelessly, at a rate of 1 error
                # per 10 characters.
                # The pseudopeople default is 1% careless.
                return {"cell_probability": 0.1, "token_probability": 0.1}
        elif noise_type == "write_wrong_digits":
            # 10% of number columns were written carelessly, at a rate of 1 error
            # per 10 characters.
            # The pseudopeople default is 1% careless.
            # Note that this is applied on top of (the default lower levels of) typos,
            # since typos also apply to numeric characters.
            return {"cell_probability": 0.1, "token_probability": 0.1}

    return default_value


def row_noise_value(dataset, noise_type, default_value):
    return default_value

In [5]:
custom_configuration = {
    dataset: {
        noise_category: (
            ({
                column: {
                    noise_type: column_noise_value(dataset, column, noise_type, noise_type_config)
                    for noise_type, noise_type_config in column_config.items()
                }
                for column, column_config in noise_category_config.items()
            }
            if noise_category == "column_noise" else
            {
                noise_type: row_noise_value(dataset, noise_type, noise_type_config)
                for noise_type, noise_type_config in noise_category_config.items()
            })
        )
        for noise_category, noise_category_config in dataset_config.items()
    }
    for dataset, dataset_config in default_configuration.items()
}

### Record ID tracking (data lineage)

We do a little bit of work here to enable tracking the "ground truth" (the simulant IDs from
pseudopeople).
We give each pseudopeople record/row a unique identifier for tracking, and then we immediately
separate the ground truth information (which we would not have if we were using real data)
from the rest of the columns (which we would have).
The ground truth is only used in the specific "ground truth" section of this notebook,
to help avoid accidentally leaking information into the case study.

Since we also combine/aggregate pseudopeople records as part of the process of generating the
simulated PVS reference files, ground truth is a bit more complicated than you might imagine.
For example, the ground truth may tell us that a single row in a reference file is actually
a composite of several individuals, because even the deterministic linkage (by SSN) we use
here is not without error.

We handle this by tracking *all* source records used in the construction of each record in
our reference files.
This is achieved by including a source_record_ids column in all files, which is a tuple
of all these records.
When we aggregate records, this is combined accordingly.

In [6]:
def add_unique_record_id(df, dataset_name):
    df = df.reset_index().rename(columns={'index': 'record_id'})
    df['record_id'] = f'{dataset_name}_' + df.record_id.astype(str)
    return df

# Initializes the source_record_ids column.
# Should only be called on "source records"; that is, records that
# come directly out of pseudopeople.
def record_id_to_single_source_record(df, source_col='record_id'):
    df = df.copy()
    df['source_record_ids'] = df[source_col].apply(lambda x: (x,))
    return df

In [7]:
# Operations that aggregate records, combining the source_record_ids column
# between all records that are aggregated into a single row

def merge_preserving_source_records(dfs, *args, **kwargs):
    dfs = [df.drop(columns=['record_id'], errors='ignore') for df in dfs]
    for df in dfs:
        assert 'source_record_ids' in df.columns

    result = dfs[0]
    for df_to_merge in dfs[1:]:
        result = (
            result.merge(df_to_merge, *args, **kwargs)
                # Get all unique source records that contributed to either record
                # Need to fill nulls here, because an outer join can cause a composite
                # record to be created only from rows in one of the dfs
                .assign(source_record_ids=lambda df: (fillna_empty_tuple(df.source_record_ids_x) + fillna_empty_tuple(df.source_record_ids_y)).apply(set).apply(tuple))
                .drop(columns=['source_record_ids_x', 'source_record_ids_y'])
        )

    return result

# Weirdly, it is quite hard to fill NaNs in a Series with
# the literal value of an empty tuple.
# See https://stackoverflow.com/a/62689667/
def fillna_empty_tuple(s):
    return s.fillna({i: tuple() for i in s.index})

import itertools

def dedupe_preserving_source_records(df, columns_to_dedupe, source_records_col='source_record_ids'):
    return (
        # NOTE: If not for dropna=False, we would silently lose
        # all rows with a null in any of the columns_to_dedupe
        df.groupby(columns_to_dedupe, dropna=False)
            # Concatenate all the tuples into one big tuple
            # https://stackoverflow.com/a/3021851/
            [source_records_col].apply(lambda x: tuple(set(itertools.chain(*x))))
            .reset_index()
    )

### SSA Numident

Wagner and Layne, p.4:

> The reference files are derived from the Social Security Administration
    (SSA) Numerical Identification file (SSA Numident). The Numident contains all
    transactions recorded against one Social Security Number (SSN)...

Based on the [SSA Numident through 2007 which is publicly available from NARA](https://aad.archives.gov/aad/series-description.jsp?s=5057),
we know there are three kinds of transactions: SSN applications, deaths, and claiming benefits.
SSN holders may change their information (e.g. changing name or sex) by submitting another application,
which generates an additional application transaction.
(The policies about this are found [on the SSA website](https://secure.ssa.gov/poms.nsf/lnx/0110212200).)

The paper ["Likely Transgender Individuals in U.S. Federal Administrative Records and the 2010 Census" by Benjamin Cerf Harris](https://www.census.gov/content/dam/Census/library/working-papers/2015/adrm/carra-wp-2015-03.pdf)
includes some helpful statistics (Table 2).
The average person in the SSA Numident has 2.2 transactions (called "claims" in that paper, but with the same definition
as our term "transaction": "Any time an SSN is created or information associated with an existing SSN is changed, that event is registered
as a claim.").

pseudopeople does not currently include correction, name change, or benefits claim transactions.
It only includes SSN creation and death of the SSN holder.

I've figured that there would be some delay in getting the Numident -- so by Census processing time
for the 2030 Census, only the SSA transactions by the end of 2029 would be available.
Note that with pseudopeople's current design it is only possible to set this cutoff at the end of a calendar year.
The NORC report says that "the Census NUMIDENT is recreated each year, to reflect
Social Security transaction records through **March** of each year" (p. 105),
though it isn't clear when in the year the Census Numident is actually re-created.

The SSA Numident is supposed to contain a sex column, but it currently doesn't in pseudopeople.

In [8]:
%%time

ssa_numident = psp.generate_social_security(year=2029, config=custom_configuration)
ssa_numident = add_unique_record_id(ssa_numident, 'ssa_numident')
ssa_numident = record_id_to_single_source_record(ssa_numident)
ssa_numident

CPU times: user 2.5 s, sys: 51.2 ms, total: 2.55 s
Wall time: 2.54 s


,record_id,simulant_id,ssn,first_name,middle_name,last_name,date_of_birth,sex,event_type,event_date,source_record_ids
0,ssa_numident_0,0_19979,786-77-6454,Evelyn,Nancy,Hinww,19191204,Female,creation,19191204,"(ssa_numident_0,)"
1,ssa_numident_1,0_6846,688-88-6377,George,Robert,Dickens,19210616,Male,creation,19211606,"(ssa_numident_1,)"
2,ssa_numident_2,0_19983,651-33-9561,Beatrise,Jennie,Fackler,19220113,Female,creation,19220113,"(ssa_numident_2,)"
3,ssa_numident_3,0_262,665-25-7858,Eura,Nadine,Crusen,19220305,Female,creation,NaN,"(ssa_numident_3,)"
4,ssa_numident_4,0_12473,875-10-2359,Roberta,Ruth,Muilenburg,19220306,Female,creation,19220306,"(ssa_numident_4,)"
...,...,...,...,...,...,...,...,...,...,...,...
19866,ssa_numident_19866,0_23602,203-10-6855,Birdie,Lizzy,Mercer,20291223,Female,creation,20291223,"(ssa_numident_19866,)"
19867,ssa_numident_19867,0_23588,505-88-8814,Levi,Holden,Small,20291224,Male,creation,20291224,"(ssa_numident_19867,)"
19868,ssa_numident_19868,0_23589,417-18-3811,Alan,Messiah,Bazan,20291227,Male,creation,20291227,"(ssa_numident_19868,)"
19869,ssa_numident_19869,0_23596,450-74-3175,Emmi,Everlynn,Will,20291228,Female,creation,20291228,"(ssa_numident_19869,)"


In [9]:
ssa_numident_ground_truth = ssa_numident.set_index('record_id').simulant_id
ssa_numident = ssa_numident.drop(columns=['simulant_id'])

### 1040 tax filings

We assume that the last 5 years of taxes would be available and used in the construction of the reference files -- see section about reference files below.

Note that these are retrieved by *tax* year, so the 2029 taxes would be available in early 2030
(around when our hypothetical case study is taking place).

In [10]:
tax_years = list(range(2025, 2030))
tax_years

[2025, 2026, 2027, 2028, 2029]

In [11]:
%%time

# Combine 1040 for all years, adding a tax_year column to track which tax year each row came from.
taxes_1040 = pd.concat([
    psp.generate_taxes_1040(year=year, config=custom_configuration).assign(tax_year=year, tax_form='1040') for year in tax_years
], ignore_index=True)
taxes_1040 = add_unique_record_id(taxes_1040, '1040')
taxes_1040 = record_id_to_single_source_record(taxes_1040)
taxes_1040

CPU times: user 30.6 s, sys: 844 ms, total: 31.5 s
Wall time: 30 s


,record_id,simulant_id,household_id,first_name,middle_initial,last_name,ssn,mailing_address_street_number,mailing_address_street_name,mailing_address_unit_number,...,dependent_2_ssn,dependent_3_first_name,dependent_3_last_name,dependent_3_ssn,dependent_4_first_name,dependent_4_last_name,dependent_4_ssn,tax_year,tax_form,source_record_ids
0,1040_0,0_5623,0_2289,Heather,J,Campusano Robledo,794-23-1522,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025,1040,"(1040_0,)"
1,1040_1,0_11911,0_10411,Gentle,J,Nicholas,004-53-9627,1934,cypress ave,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025,1040,"(1040_1,)"
2,1040_2,0_25,0_16,Heather,S,Blackwell,273-52-7750,57,shadowood dr,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025,1040,"(1040_2,)"
3,1040_3,0_15029,0_6055,Candy,S,Luedeman,384-45-8523,10881,maple av,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025,1040,"(1040_3,)"
4,1040_4,0_18706,0_7520,Patrick,J,Morfin,858-84-4517,103,71st st w,NaN,...,691-64-4425,Nathan,Morfin,539-57-3727,NaN,NaN,NaN,2025,1040,"(1040_4,)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39114,1040_39114,0_21814,0_4732,Akari,L,Parkinson,595-04-6651,2921,curtis la,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2029,1040,"(1040_39114,)"
39115,1040_39115,0_22705,0_11188,Jada,T,Franco Resendez,523-63-9324,1920,white ibis ct,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2029,1040,"(1040_39115,)"
39116,1040_39116,0_16072,0_6479,Bianca,L,Sargent,187-01-0690,12613,teppert,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2029,1040,"(1040_39116,)"
39117,1040_39117,0_23238,0_11927,Ashton,N,Cardenas,808-04-3639,116,wakema rd,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2029,1040,"(1040_39117,)"


In [12]:
taxes_1040_ground_truth = taxes_1040.set_index('record_id').simulant_id
taxes_1040 = taxes_1040.drop(columns=['simulant_id'])

### W2/1099 tax filings

We assume that the last 5 years of taxes would be available and used in the construction of the reference files --
see section about reference files below.

Note that these are retrieved by *tax* year, so the 2029 taxes would be available in early 2030
(around when our hypothetical case study is taking place).

In [13]:
%%time

# Combine W2/1099 for all years, adding a tax_year column to track which tax year each row came from.
w2_1099 = pd.concat([
    psp.generate_taxes_w2_and_1099(year=year, config=custom_configuration).assign(tax_year=year) for year in tax_years
], ignore_index=True)
w2_1099 = add_unique_record_id(w2_1099, 'w2_1099')
w2_1099 = record_id_to_single_source_record(w2_1099)
w2_1099

CPU times: user 25.8 s, sys: 682 ms, total: 26.5 s
Wall time: 25.5 s


,record_id,simulant_id,household_id,employer_id,ssn,wages,employer_name,employer_street_number,employer_street_name,employer_unit_number,...,mailing_address_street_number,mailing_address_street_name,mailing_address_unit_number,mailing_address_po_box,mailing_address_city,mailing_address_state,mailing_address_zipcode,tax_form,tax_year,source_record_ids
0,w2_1099_0,0_5,0_8,12,854-13-6295,27185,Jj Rubys Salon Studios,1300,windsor lane,NaN,...,1501,interlake ave n,NaN,NaN,Anytown,WA,00000,W2,2025,"(w2_1099_0,)"
1,w2_1099_1,0_5,0_8,60,854-13-6295,7920,Freeway Insurance Agency,1105,largess ln,NaN,...,1501,interlake ave n,NaN,NaN,Anytown,WA,00000,W2,2025,"(w2_1099_1,)"
2,w2_1099_2,0_5623,0_2289,11,794-23-1522,95715,Prenatal,10949,w diamond bell ranch rd,NaN,...,NaN,NaN,NaN,14011,Anytown,WA,00000,W2,2025,"(w2_1099_2,)"
3,w2_1099_3,0_7252,0_2957,69,093-85-3512,1616,Rancho Vistoso Trails Mental Health,2101,w 4th pl,NaN,...,927,broomfield ln,NaN,NaN,Anytown,WA,00000,W2,2025,"(w2_1099_3,)"
4,w2_1099_4,0_7252,0_2957,95,093-85-3512,2683,Pikes Creek Campground,6300,east lincoln ave,NaN,...,927,broomfield ln,NaN,NaN,Anytown,WA,00000,W2,2025,"(w2_1099_4,)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53619,w2_1099_53619,0_8910,0_3621,53,119-02-0252,121064,A Car Title Loans,6100,e ball rd,NaN,...,3515,javello rd,NaN,NaN,Anytown,WA,00000,NaN,2029,"(w2_1099_53619,)"
53620,w2_1099_53620,0_2378,0_12207,27,079-75-0734,34645,The Wicked Pho Licious Burgers,108,normandy lane,NaN,...,9358,e 58th st,NaN,NaN,Anytown,WA,00000,W2,2029,"(w2_1099_53620,)"
53621,w2_1099_53621,0_2378,0_12207,54,079-75-0734,NaN,BP,4523,lawrence rd,NaN,...,9358,e 58th st,NaN,NaN,Anytown,WA,00000,W2,2029,"(w2_1099_53621,)"
53622,w2_1099_53622,0_22909,0_11487,30,z87-19-2431,41695,Warrensburg,1830,kittery pt,NaN,...,NaN,rdg park dr,NaN,NaN,Anytown,WA,00000,W2,2029,"(w2_1099_53622,)"


In [14]:
w2_1099_ground_truth = w2_1099.set_index('record_id').simulant_id
w2_1099 = w2_1099.drop(columns=['simulant_id'])

In [15]:
taxes = pd.concat([taxes_1040, w2_1099], ignore_index=True)

In [16]:
taxes_ground_truth = pd.concat([taxes_1040_ground_truth, w2_1099_ground_truth])

In [17]:
# "... many of the [IRS] records contain only the first four letters of the last name."
# (Brown et al. 2023, p.30, footnote 19)
# This should be updated in pseudopeople but for now we do it here.
# Note that this truncation only matters for ITIN PIKing since for SSNs that are present in SSA we use name from SSA.
PROPORTION_OF_IRS_RECORDS_WITH_TRUNCATION = 0.4 # is this a good guess at "many" in the quote above?
idx_to_truncate = taxes.sample(frac=PROPORTION_OF_IRS_RECORDS_WITH_TRUNCATION, random_state=1234).index
taxes.loc[idx_to_truncate, 'last_name'] = taxes.loc[idx_to_truncate, 'last_name'].str[:4]
taxes.loc[idx_to_truncate, 'last_name']

76856    Lang
62755    Wile
45       Magw
80316    Smit
73427    Hami
         ... 
67877    Hebe
47283    Hans
81811    Thru
44934    Ring
44293    Kenn
Name: last_name, Length: 37097, dtype: object

### 2030 Census Unedited File (CUF)

For now, we gloss over the data schema for addresses.
We don't know how addresses would be formatted in the CUF (and it's hard to guess, because
address is not part of the Census form), but it likely would have some of these fields
(street number, street name, etc) combined.

While PVS input files do not in general have names split into first, middle, and last,
I am guessing the CUF **would** have first name, middle initial, last name (which is how pseudopeople
generates it), because that [matches the Census questionnaire](https://www2.census.gov/programs-surveys/decennial/2020/technical-documentation/questionnaires-and-instructions/questionnaires/2020-informational-questionnaire-english_DI-Q1.pdf).

In [18]:
%%time

census_2030 = psp.generate_decennial_census(year=2030, config=custom_configuration)
census_2030 = add_unique_record_id(census_2030, 'census_2030')
census_2030 = record_id_to_single_source_record(census_2030)
census_2030

CPU times: user 3.88 s, sys: 88.4 ms, total: 3.97 s
Wall time: 3.9 s


,record_id,simulant_id,household_id,first_name,middle_initial,last_name,age,date_of_birth,street_number,street_name,unit_number,city,state,zipcode,housing_type,relationship_to_reference_person,sex,race_ethnicity,year,source_record_ids
0,census_2030_0,0_923,0_8033,Gerald,R,Allen,86,11/03/1943,1130,mallory ln,NaN,Anytown,WA,00000,Household,Reference person,Male,Black,2030,"(census_2030_0,)"
1,census_2030_1,0_2348,0_1066,April,S,Hayden,33,10/23/1996,32597,delacorte dr,NaN,Anytown,WA,00000,Household,Other nonrelative,Female,Black,2030,"(census_2030_1,)"
2,census_2030_2,0_2641,0_1066,Loretta,T,Lowe,71,06/01/1958,32597,delacorte dr,NaN,Anytown,WA,00000,Household,Reference person,Female,White,2030,"(census_2030_2,)"
3,census_2030_3,0_6176,0_2514,Sandra,S,Sorrentino,75,03/18/1954,4458,windsor pl,NaN,Anytown,WA,00000,Household,NaN,Female,Multiracial or Other,2030,"(census_2030_3,)"
4,census_2030_4,0_10251,0_5627,Bobby,S,Baker,54,05/20/1985,17868,winding trail rd,NaN,Anytown,WA,00000,Household,Other nonrelative,Male,White,2030,"(census_2030_4,)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11018,census_2030_11018,0_17003,0_10778,Jeremy,Y,Boyd,46,07/01/1983,211,quiet way,NaN,Anytown,WA,00000,Household,Reference person,Male,Black,2030,"(census_2030_11018,)"
11019,census_2030_11019,0_19380,0_11001,Wendy,M,Gross,54,12/05/1975,2801,blue rdg dr n,NaN,Anytown,WA,00000,Household,Reference person,Female,White,2030,"(census_2030_11019,)"
11020,census_2030_11020,0_20272,0_5308,Ember,H,Samuels,10,10/26/2019,24113,lauder,NaN,Anytown,WA,00000,Household,Reference person,Female,Black,2030,"(census_2030_11020,)"
11021,census_2030_11021,0_21997,0_10693,Athena,V,Deshpande,27,07/05/2002,1534,bentley dr,NaN,Anytown,WA,00000,Household,Reference person,NaN,Asian,2030,"(census_2030_11021,)"


In [19]:
census_2030_ground_truth = census_2030.set_index('record_id').simulant_id
census_2030 = census_2030.drop(columns=['simulant_id'])

In [20]:
source_record_ground_truth = pd.concat([
    ssa_numident_ground_truth,
    taxes_ground_truth,
    census_2030_ground_truth,
])
source_record_ground_truth

record_id
ssa_numident_0       0_19979
ssa_numident_1        0_6846
ssa_numident_2       0_19983
ssa_numident_3         0_262
ssa_numident_4       0_12473
                      ...   
census_2030_11018    0_17003
census_2030_11019    0_19380
census_2030_11020    0_20272
census_2030_11021    0_21997
census_2030_11022    0_23386
Name: simulant_id, Length: 123637, dtype: object

## Create reference files

> The Census Numident – all Social Security Administration (SSA) Numident SSN records are
  edited (collapsed) to produce a Census Numident file that contains “one best-data record” for
  each SSN. All variants of name information for each SSN are retained in the Alternate Name
  Numident file, while all variants of date of birth data are retained in the Alternate DOB
  Numident. The SSN-PIK crosswalk file is used to attach a corresponding unique PIK value for
  each SSN value in the Census Numident file.

### Census Numident

Luque and Wagner, p. 4:
  
> The SSA Numident file contains all transactions ever recorded against any single SSN - with each entry
representing an addition or change (such as name changes) to the SSN record. This file is edited to
create the **Census Numident**, which contains one record for each SSN. Each SSN record in the Census
Numident contains name, DOB, sex, race, place of birth, parents’ name, citizenship status and date of death information.

and in footnote 5:

> Name edits, DOB reconciliation, and race identifiers are some of the edits conducted to produce this Numident
file. **The resulting Numident file contains the most recent name and DOB data.**

We are missing quite a few columns, since they are missing in pseudopeople's SSA Numident: sex, race, place of birth, parents' name,
citizenship status.
However, I'm pretty sure that out of these columns, only sex is used in the PVS linking process.

In [21]:
def fill_dates(df, fill_with):
    return (
        # Replace invalid dates with nans
        pd.to_datetime(df.event_date, format='%Y%m%d', errors='coerce')
            .fillna(pd.to_datetime('2100-01-01' if fill_with == 'latest' else '1900-01-01'))
    )

def best_data_from_columns(df, columns, best_is_latest=True):
    # We don't want to throw out events with a missing/invalid date, so we'll fill them with the value *least* likely to be chosen
    # (earlier than all values if taking the latest, later than all values if taking the earliest).
    fill_with = 'earliest' if best_is_latest else 'latest'

    return (
        df
            # Without mutating the existing date column, get one that is actually
            # a date type and can be used for sorting.
            .assign(event_date_for_sort=lambda df: fill_dates(df, fill_with=fill_with))
            .sort_values('event_date_for_sort')
            .dropna(subset=columns, how='all')
            .drop_duplicates('ssn', keep=('last' if best_is_latest else 'first'))
            [['record_id', 'ssn'] + columns]
            .pipe(record_id_to_single_source_record)
    )

best_name = best_data_from_columns(
    ssa_numident,
    columns=['first_name', 'middle_name', 'last_name'],
)

best_date_of_birth = best_data_from_columns(
    ssa_numident,
    columns=['date_of_birth'],
)

best_date_of_death = best_data_from_columns(
    ssa_numident[ssa_numident.event_type == 'death'],
    columns=['event_date'],
).rename(columns={'event_date': 'date_of_death'})

census_numident = merge_preserving_source_records(
    [best_name, best_date_of_birth, best_date_of_death],
    on='ssn',
    how='outer',
)
census_numident = add_unique_record_id(census_numident, 'census_numident')
census_numident

,record_id,ssn,first_name,middle_name,last_name,date_of_birth,date_of_death,source_record_ids
0,census_numident_0,565-46-1886,Salbador,Robert,NaN,19780317,NaN,"(ssa_numident_6730,)"
1,census_numident_1,221-20-4546,Rebekah,Kandace,Woo,19910627,NaN,"(ssa_numident_9511,)"
2,census_numident_2,116-26-7703,Daniel,Michael,Hernandez,19870904,NaN,"(ssa_numident_8710,)"
3,census_numident_3,594-13-8945,Kelly,Ashley,Safarian,19911110,NaN,"(ssa_numident_9585,)"
4,census_numident_4,702-03-1779,Dalton,Jordan,Ybarra Canalfs,19980818,NaN,"(ssa_numident_11036,)"
...,...,...,...,...,...,...,...,...
18638,census_numident_18638,381-54-9050,Luke,Devin,Blackman,20230502,NaN,"(ssa_numident_17366,)"
18639,census_numident_18639,154-06-3393,Gary,Ronald,Palumbo,19520217,22240611,"(ssa_numident_17778,)"
18640,census_numident_18640,542-88-1464,John,Victor,Skeeter,19611229,22260313,"(ssa_numident_18446,)"
18641,census_numident_18641,671-15-8285,Leighton,Sophia,Davis,20280207,NaN,"(ssa_numident_19146,)"


### Alternate Name Numident

Wagner and Layne, p. 9:

>  All variants of name information for each SSN are retained in the Alternate Name
Numident file...

In [22]:
alternate_name_numident = dedupe_preserving_source_records(ssa_numident, ['ssn', 'first_name', 'middle_name', 'last_name'])
alternate_name_numident = add_unique_record_id(alternate_name_numident, 'alternate_name_numident')
alternate_name_numident

,record_id,ssn,first_name,middle_name,last_name,source_record_ids
0,alternate_name_numident_0,001-02-4588,Ethan,Michael,Boyadjian,"(ssa_numident_13580,)"
1,alternate_name_numident_1,001-11-6330,Tammy,Pamela,Roop,"(ssa_numident_6350,)"
2,alternate_name_numident_2,001-16-0077,Gena,Allison,Mathews,"(ssa_numident_5155,)"
3,alternate_name_numident_3,001-17-9511,Heather,Cathy,Gunter,"(ssa_numident_4501,)"
4,alternate_name_numident_4,001-25-8258,Apollo,Kyro,Ortega,"(ssa_numident_18591,)"
...,...,...,...,...,...,...
19062,alternate_name_numident_19062,989-21-7032,Ethan,Gianni,Jones,"(ssa_numident_9997,)"
19063,alternate_name_numident_19063,990-97-7953,Jason,David,Jones,"(ssa_numident_7873,)"
19064,alternate_name_numident_19064,992-40-7512,Ronald,Walter,Sciulli,"(ssa_numident_17091,)"
19065,alternate_name_numident_19065,997-52-5016,Melissa,Ashley,Haggerty,"(ssa_numident_7220,)"


In [23]:
alternate_name_numident.groupby('ssn').size().describe()

count    18643.000000
mean         1.022743
std          0.149087
min          1.000000
25%          1.000000
50%          1.000000
75%          1.000000
max          2.000000
dtype: float64

In [24]:
alternate_name_numident[alternate_name_numident.ssn.duplicated(keep=False)].sort_values('ssn')

,record_id,ssn,first_name,middle_name,last_name,source_record_ids
5,alternate_name_numident_5,001-30-0491,Jacqueline,Mary,Gomez,"(ssa_numident_1187,)"
6,alternate_name_numident_6,001-30-0491,Jacqueline,NaN,Gomez,"(ssa_numident_19683,)"
48,alternate_name_numident_48,003-21-2342,Richard,Richard,Bergeron,"(ssa_numident_16452,)"
49,alternate_name_numident_49,003-21-2342,Richard,Richie,Bergeron,"(ssa_numident_849,)"
67,alternate_name_numident_67,003-84-2583,Diane,Jeamne,Coleman,"(ssa_numident_3393,)"
...,...,...,...,...,...,...
19015,alternate_name_numident_19015,898-96-4862,Shirlfy,Barbara,Burgos-Valdez,"(ssa_numident_17910,)"
19017,alternate_name_numident_19017,899-07-0109,Jeffrey,Gleng,Hawkins,"(ssa_numident_17752,)"
19018,alternate_name_numident_19018,899-07-0109,Jeffrey,Glenn,Hawkins,"(ssa_numident_2971,)"
19022,alternate_name_numident_19022,899-24-9400,Fred,David,Laduke,"(ssa_numident_2140,)"


### Alternate DOB Numident

Wagner and Layne, p. 9:

> ... while all variants of date of birth data are retained in the Alternate DOB
Numident.

In [25]:
alternate_dob_numident = dedupe_preserving_source_records(ssa_numident, ['ssn', 'date_of_birth'])
alternate_dob_numident = add_unique_record_id(alternate_dob_numident, 'alternate_dob_numident')
alternate_dob_numident

,record_id,ssn,date_of_birth,source_record_ids
0,alternate_dob_numident_0,001-02-4588,20080901,"(ssa_numident_13580,)"
1,alternate_dob_numident_1,001-11-6330,19760402,"(ssa_numident_6350,)"
2,alternate_dob_numident_2,001-16-0077,19700315,"(ssa_numident_5155,)"
3,alternate_dob_numident_3,001-17-9511,19661109,"(ssa_numident_4501,)"
4,alternate_dob_numident_4,001-25-8258,20260816,"(ssa_numident_18591,)"
...,...,...,...,...
18882,alternate_dob_numident_18882,989-21-7032,19931105,"(ssa_numident_9997,)"
18883,alternate_dob_numident_18883,990-97-7953,19831016,"(ssa_numident_7873,)"
18884,alternate_dob_numident_18884,992-40-7512,19380304,"(ssa_numident_17091,)"
18885,alternate_dob_numident_18885,997-52-5016,19800917,"(ssa_numident_7220,)"


In [26]:
alternate_dob_numident.groupby('ssn').size().describe()

count    18643.000000
mean         1.013088
std          0.113655
min          1.000000
25%          1.000000
50%          1.000000
75%          1.000000
max          2.000000
dtype: float64

In [27]:
alternate_dob_numident[alternate_dob_numident.ssn.duplicated(keep=False)].sort_values('ssn')

,record_id,ssn,date_of_birth,source_record_ids
111,alternate_dob_numident_111,005-58-4870,19561009,"(ssa_numident_19751,)"
112,alternate_dob_numident_112,005-58-4870,19661009,"(ssa_numident_4485,)"
247,alternate_dob_numident_247,013-16-8392,12680418,"(ssa_numident_17314,)"
248,alternate_dob_numident_248,013-16-8392,19680818,"(ssa_numident_4870,)"
249,alternate_dob_numident_249,013-17-9455,12470921,"(ssa_numident_17998,)"
...,...,...,...,...
18704,alternate_dob_numident_18704,892-75-9447,19290519,"(ssa_numident_17217,)"
18723,alternate_dob_numident_18723,893-51-7195,09580101,"(ssa_numident_15803,)"
18724,alternate_dob_numident_18724,893-51-7195,19580101,"(ssa_numident_2834,)"
18808,alternate_dob_numident_18808,897-75-4542,13850509,"(ssa_numident_18553,)"


### Name/DOB Reference File

Wagner and Layne, p. 9:

> The Name and DOB Reference files are reformatted versions of the Census Numident
and includes **all possible combinations of alternate names and dates of birth, as well as
ITIN data**. All of the reference files contain SSN/ITIN and the corresponding PIK. When
an input record is linked to a reference file, the corresponding PIK is assigned. Table 1
presents the number of observations in each of the reference files.

A slightly confusing point: sometimes the Name and DOB reference files are described
as one and the same thing, and sometimes as separate.
I believe this is because **they differ only in how they are "cut" for the PVS process:**
the name reference file is cut by first and last initial,
while the DOB reference file is cut by month and day of birth.

This is described in Wagner and Layne, p.15:

> The [DOBSearch] module matches against a re-split
version of the Numident Name Reference file, splitting the data based on month and day
of birth.

Since we handle the logic of "cutting" in the linkage process itself, we generate
a single reference file here.

Note that unlike for addresses, and unlike for the pre-processing of PVS *input* files
(as opposed to reference files), there is no explicit nickname processing/correction here.
I am fairly sure that is accurate to the real PVS, which I believe assumes that nicknames
would not be present in SSA/tax records (or at least, that the real name would appear
at least once in these records).

In [28]:
name_dob_numident_records = merge_preserving_source_records(
    [alternate_name_numident, alternate_dob_numident],
    on='ssn',
    how='outer',
)
name_dob_numident_records

,ssn,first_name,middle_name,last_name,date_of_birth,source_record_ids
0,001-02-4588,Ethan,Michael,Boyadjian,20080901,"(ssa_numident_13580,)"
1,001-11-6330,Tammy,Pamela,Roop,19760402,"(ssa_numident_6350,)"
2,001-16-0077,Gena,Allison,Mathews,19700315,"(ssa_numident_5155,)"
3,001-17-9511,Heather,Cathy,Gunter,19661109,"(ssa_numident_4501,)"
4,001-25-8258,Apollo,Kyro,Ortega,20260816,"(ssa_numident_18591,)"
...,...,...,...,...,...,...
19382,989-21-7032,Ethan,Gianni,Jones,19931105,"(ssa_numident_9997,)"
19383,990-97-7953,Jason,David,Jones,19831016,"(ssa_numident_7873,)"
19384,992-40-7512,Ronald,Walter,Sciulli,19380304,"(ssa_numident_17091,)"
19385,997-52-5016,Melissa,Ashley,Haggerty,19800917,"(ssa_numident_7220,)"


In [29]:
name_dob_numident_records[name_dob_numident_records.ssn.duplicated(keep=False)].sort_values('ssn')

,ssn,first_name,middle_name,last_name,date_of_birth,source_record_ids
5,001-30-0491,Jacqueline,Mary,Gomez,19460824,"(ssa_numident_1187, ssa_numident_19683)"
6,001-30-0491,Jacqueline,NaN,Gomez,19460824,"(ssa_numident_1187, ssa_numident_19683)"
48,003-21-2342,Richard,Richard,Bergeron,19430622,"(ssa_numident_849, ssa_numident_16452)"
49,003-21-2342,Richard,Richie,Bergeron,19430622,"(ssa_numident_849, ssa_numident_16452)"
67,003-84-2583,Diane,Jeamne,Coleman,19610314,"(ssa_numident_3393, ssa_numident_17672)"
...,...,...,...,...,...,...
19335,898-96-4862,Shirlfy,Barbara,Burgos-Valdez,19240302,"(ssa_numident_26, ssa_numident_17910)"
19337,899-07-0109,Jeffrey,Gleng,Hawkins,19580911,"(ssa_numident_17752, ssa_numident_2971)"
19338,899-07-0109,Jeffrey,Glenn,Hawkins,19580911,"(ssa_numident_17752, ssa_numident_2971)"
19342,899-24-9400,Fred,David,Laduke,19530914,"(ssa_numident_2140, ssa_numident_18365)"


#### Incorporating people with ITINs

Individual Taxpayer Identification Numbers (ITINs) can be issued to people who are required to file
federal taxes but are not eligible for a Social Security Number.
The most common reason for this is being an undocumented immigrant and therefore not being authorized
to work in the United States.

People without SSNs used to be impossible to assign PIKs to.
In 2011 the NORC report stated (p. 38, footnote 19):

> NORC understands that the Census Bureau has undertaken an effort to enhance the PVS reference files with IRS
files that include Individual Taxpayer Identification Numbers (ITIN). For those people who are required to file a tax
return but do not have, and may not want an SSN—such as a non-U.S. citizen—the IRS issues the taxpayer an ITIN.
This enhancement to the PVS reference file may help to match more non-U.S citizens.

By 2014 (Wagner and Layne, p. 5):

> One of the key enhancements [made in recent years] increased the coverage of the reference files by
including records for persons with Individual Taxpayer Identification Numbers assigned
by the Internal Revenue Service (ITINs) to [along with?] the SSN-based Numident data. 

I have not found a specific description of how ITIN records are constructed in any of the
publicly-available sources.
This may be because it is straightforward, or because the tax data schema is confidential.
I assume that only IRS data is used, since no other data source that I am aware of would
report ITIN.

It is stated that the ITIN records are created directly from tax filings and not
from ITIN applications (Brown et al. p. 29, footnote 16), which is convenient
because the tax filing data is what we can simulate with
pseudopeople:

> The NUMIDENT provides the PII on the SSN-holder from the issuing agency (SSA), and that PII is used in SSN
verification. **For ITINs, the Census Bureau does not have access to the ITIN applications** to the issuing agency (IRS),
so name and DOB verification of ITINs is less reliable.

"Less reliable" is a bit confusing here, because as stated above when generating
the simulated tax data, IRS data should not contain date of birth at all.
Here, we have stayed true to this by omitting it entirely.

We assume that only 1040 filings would be used for this purpose; we wouldn't expect ITINs to
show up on employer-filed W-2/1099 forms.

In [30]:
# Analogous to the process of getting alternate names and dates of birth
# from SSA, we retain all versions of the name from taxes.
name_for_itins = dedupe_preserving_source_records(
    taxes_1040[taxes_1040.ssn.notnull() & taxes_1040.ssn.str.startswith('9')],
    ['ssn', 'first_name', 'middle_initial', 'last_name'],
)
name_for_itins

,ssn,first_name,middle_initial,last_name,source_record_ids
0,900-54-7842,Megan,C,Kelley,"(1040_34170, 1040_18143)"
1,900-88-9530,Matthew,S,Johnson,"(1040_35087, 1040_3675, 1040_26996, 1040_11310..."
2,900-97-3510,Kevin,V,Sevy,"(1040_18486, 1040_34011, 1040_25936)"
3,901-63-3466,Kimberly,S,Cruz,"(1040_3697, 1040_27026)"
4,901-96-9924,Victoria,R,Butler,"(1040_16070,)"
...,...,...,...,...,...
622,999-92-3977,Matthew,M,Carmona,"(1040_35636,)"
623,999-99-9858,Kathryn,T,Mcmiller,"(1040_11353, 1040_19113)"
624,9s6-77-5983,Kevin,T,Ventura,"(1040_25139,)"
625,9zs-So-7561,Anna,K,Garcia Jones,"(1040_23571,)"


In [31]:
name_for_itins.groupby('ssn').size().describe()

count    509.000000
mean       1.231827
std        0.462457
min        1.000000
25%        1.000000
50%        1.000000
75%        1.000000
max        4.000000
dtype: float64

In [32]:
# Make sure these are disjoint sets of SSN values -- this could only happen
# if an SSN value in the Numident were corrupted into the ITIN range
assert set(name_dob_numident_records.ssn) & set(name_for_itins.ssn) == set()

In [33]:
name_dob_reference_file = pd.concat([
    name_dob_numident_records,
    name_for_itins,
], ignore_index=True)
name_dob_reference_file = add_unique_record_id(name_dob_reference_file, 'name_dob_reference_file')
name_dob_reference_file

,record_id,ssn,first_name,middle_name,last_name,date_of_birth,source_record_ids,middle_initial
0,name_dob_reference_file_0,001-02-4588,Ethan,Michael,Boyadjian,20080901,"(ssa_numident_13580,)",NaN
1,name_dob_reference_file_1,001-11-6330,Tammy,Pamela,Roop,19760402,"(ssa_numident_6350,)",NaN
2,name_dob_reference_file_2,001-16-0077,Gena,Allison,Mathews,19700315,"(ssa_numident_5155,)",NaN
3,name_dob_reference_file_3,001-17-9511,Heather,Cathy,Gunter,19661109,"(ssa_numident_4501,)",NaN
4,name_dob_reference_file_4,001-25-8258,Apollo,Kyro,Ortega,20260816,"(ssa_numident_18591,)",NaN
...,...,...,...,...,...,...,...,...
20009,name_dob_reference_file_20009,999-92-3977,Matthew,NaN,Carmona,NaN,"(1040_35636,)",M
20010,name_dob_reference_file_20010,999-99-9858,Kathryn,NaN,Mcmiller,NaN,"(1040_11353, 1040_19113)",T
20011,name_dob_reference_file_20011,9s6-77-5983,Kevin,NaN,Ventura,NaN,"(1040_25139,)",T
20012,name_dob_reference_file_20012,9zs-So-7561,Anna,NaN,Garcia Jones,NaN,"(1040_23571,)",K


### GeoBase Reference File

Wagner and Layne, p. 9:

> PVS creates three other sets of reference
files containing Numident data: the **GeoBase Reference File**, the Name Reference File,
and the DOB Reference file.
The GeoBase Reference File appends addresses from administrative records attached
to Numident data, including all possible combinations of alternate names and dates of
birth for SSN. Addresses from administrative records are edited and processed through
commercial software product to clean and standardize address data. ITIN data is also
incorporated into the Geobase.

Luque and Wagner, p. 5:

> Reference files contain data from the Numident file enhanced with address
data obtained from federal AR [administrative records] files.<sup>8</sup>
The reference files, thus, contain all variants of a person’s name, DOB,
and sex, as well as current and recent addresses. These reference files are
referred to as the (PVS) Geobase reference file since addresses (a geographic component)
are appended to each person record.<sup>9</sup> It is important to note that there are
multiple Geobase reference files that are created depending on the vintage of the
incoming file to be processed through PVS.

> <sup>8</sup> Namely, data from the IRS, Department of Housing and Urban Development,
several files from the Department of Health and Human Services, and Selective Service.

> <sup>9</sup> In particular, the address data is cleaned and standardized and used
to construct a variable called GEOKEY. The GEOKEY variable is constructed as a subset
of the full address, and then is appended to the Numident data to create the
PVS Geobase Reference file.

We only have IRS data to use for addresses, and specifically only W-2/1099 data,
which is a limitation of this case study.
I can't find a concrete definition of "recent" -- as noted above, we use 5 years
of IRS data.
This may be longer than the true window, but this may end up making up for
the lack of non-IRS data sources, and get us closer to a realistic number of
alternate addresses.

Also, our address data comes out of pseudopeople already parsed into address parts
like street name, etc.
For more realism, pseudopeople should output a single string that we have to (imperfectly) parse apart.

I haven't been able to find out more about what kind of "subset" the geokey is.
It is unclear to me why geokey is "interesting" since it is just derived from the
address parts.

In [34]:
address_cols = list(taxes.filter(like='mailing_address').columns)

def standardize_address_part(column):
    return (
        column
            # Remove leading or trailing whitespace
            .str.strip()
            # Turn any strings of consecutive whitespace into a single space
            .str.split().str.join(' ')
            # Normalize case
            .str.upper()
            # Normalize the word street as described in the example quoted above
            # In reality, there would be many rules like this
            .str.replace('\b(STREET|STR)\b', 'ST', regex=True)
            # Make sure missingness is represented consistently
            .replace('', np.nan)
    )

addresses_by_ssn = dedupe_preserving_source_records(
    taxes.set_index(['ssn', 'source_record_ids'])
        [address_cols]
        .apply(standardize_address_part)
        .reset_index(),
    ['ssn'] + address_cols,
)
addresses_by_ssn

,ssn,mailing_address_street_number,mailing_address_street_name,mailing_address_unit_number,mailing_address_po_box,mailing_address_city,mailing_address_state,mailing_address_zipcode,source_record_ids
0,000-79-9923,9529,COURTLAND STREET NORTHEAST,NaN,NaN,ANYTOWN,CO,00000,"(1040_18528,)"
1,001-15-8330,13024,RHOADS AVE,NaN,NaN,ANYTOWN,WA,00000,"(w2_1099_39059, w2_1099_49819, 1040_24966, w2_..."
2,001-16-0067,60,EGGLESTON RD,NaN,NaN,ANYTOWN,WA,00000,"(w2_1099_48364,)"
3,001-16-0077,60,EGGLESTON RD,NaN,NaN,ANYTOWN,WA,00000,"(w2_1099_37615, 1040_16507, 1040_1366, w2_1099..."
4,001-16-0077,60,NaN,NaN,NaN,ANYTOWN,WA,00000,"(1040_8905,)"
...,...,...,...,...,...,...,...,...,...
32567,NaN,NaN,NaN,NaN,8089,ANYTOWN,WA,00000,"(w2_1099_39989,)"
32568,NaN,NaN,NaN,NaN,9141,ANYTOWN,WA,00000,"(1040_28468,)"
32569,NaN,NaN,NaN,NaN,9483,ANYTOWN,WA,00000,"(w2_1099_43007,)"
32570,NaN,NaN,NaN,NaN,9640,ANYTOWN,WA,00000,"(w2_1099_37638,)"


In [35]:
num_addresses = addresses_by_ssn.groupby('ssn').size().sort_values()
num_addresses

ssn
000-79-9923    1
545-28-0443    1
545-31-3602    1
545-35-4995    1
545-80-0380    1
              ..
543-85-7977    7
682-87-8833    7
347-72-4591    7
056-61-5935    7
144-18-4623    9
Length: 19291, dtype: int64

In [36]:
# Show some SSNs with a lot of address variation
addresses_by_ssn[addresses_by_ssn.ssn.isin(num_addresses.tail(10).index)].sort_values('ssn')

,ssn,mailing_address_street_number,mailing_address_street_name,mailing_address_unit_number,mailing_address_po_box,mailing_address_city,mailing_address_state,mailing_address_zipcode,source_record_ids
275,007-67-7210,112,WESTERN AVE,NaN,NaN,ANYTOWN,WA,00000,"(w2_1099_10387,)"
276,007-67-7210,112,WSTTERN AV,NaN,NaN,ANYTOWN,WA,00000,"(w2_1099_10386,)"
277,007-67-7210,24,BOWEN CIR SW,NaN,NaN,ANYTOWN,WA,00000,"(w2_1099_43135,)"
278,007-67-7210,34,BOWEN CIR SW,NaN,NaN,ANYTOWN,WA,00000,"(w2_1099_11025, w2_1099_11026, w2_1099_21522, ..."
279,007-67-7210,34,BOWEN CIR SW,NaN,NaN,ANYTOWN,WA,00003,"(w2_1099_32387,)"
...,...,...,...,...,...,...,...,...,...
29640,859-49-8970,1608,W NORT AVENUE,NaN,NaN,ANYTOWN,WA,00000,"(1040_16231, 1040_8634, w2_1099_26052, w2_1099..."
29641,859-49-8970,1608,W NORT AVENUE,NaN,NaN,ANYTOWN,WA,NaN,"(w2_1099_4783,)"
29642,859-49-8970,1608,W NORT AVENUE,NaN,NaN,ANYTOWN,NaN,00000,"(w2_1099_4784,)"
29643,859-49-8970,69,S HWY 701,NaN,NaN,ANYTOWH,WA,NaN,"(w2_1099_45343,)"


In [37]:
# Rough estimate of how many rows we should have in our reference file, once we do this Cartesian product
(
    len(name_dob_reference_file) *
    addresses_by_ssn.groupby('ssn').size().mean()
)

33018.79446373956

In [38]:
geobase_reference_file = merge_preserving_source_records(
    [name_dob_reference_file, addresses_by_ssn],
    on='ssn',
    how='left',
)
geobase_reference_file = add_unique_record_id(geobase_reference_file, 'geobase_reference_file')
geobase_reference_file

,record_id,ssn,first_name,middle_name,last_name,date_of_birth,middle_initial,mailing_address_street_number,mailing_address_street_name,mailing_address_unit_number,mailing_address_po_box,mailing_address_city,mailing_address_state,mailing_address_zipcode,source_record_ids
0,geobase_reference_file_0,001-02-4588,Ethan,Michael,Boyadjian,20080901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(ssa_numident_13580,)"
1,geobase_reference_file_1,001-11-6330,Tammy,Pamela,Roop,19760402,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(ssa_numident_6350,)"
2,geobase_reference_file_2,001-16-0077,Gena,Allison,Mathews,19700315,NaN,60,EGGLESTON RD,NaN,NaN,ANYTOWN,WA,00000,"(w2_1099_37615, 1040_16507, 1040_1366, w2_1099..."
3,geobase_reference_file_3,001-16-0077,Gena,Allison,Mathews,19700315,NaN,60,NaN,NaN,NaN,ANYTOWN,WA,00000,"(ssa_numident_5155, 1040_8905)"
4,geobase_reference_file_4,001-17-9511,Heather,Cathy,Gunter,19661109,NaN,575,18TH ST,NaN,NaN,ANYTOWN,WA,00000,"(ssa_numident_4501, w2_1099_41088, w2_1099_304..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32182,geobase_reference_file_32182,999-92-3977,Matthew,NaN,Carmona,NaN,M,4000,NS SKYCREST PKWY,NaN,NaN,ANYTOWN,WA,00000,"(1040_35636,)"
32183,geobase_reference_file_32183,999-99-9858,Kathryn,NaN,Mcmiller,NaN,T,1231,RIVERSIDE DR,NaN,NaN,ANYTOWN,WA,00000,"(1040_11353, 1040_19113)"
32184,geobase_reference_file_32184,9s6-77-5983,Kevin,NaN,Ventura,NaN,T,56,LANCASTER ST,NaN,NaN,ANYTOWN,WA,00000,"(1040_25139,)"
32185,geobase_reference_file_32185,9zs-So-7561,Anna,NaN,Garcia Jones,NaN,K,NaN,W GREENFIELD AVE,NaN,NaN,ANYTOWN,WA,00000,"(1040_23571,)"


## Track ground truth for reference files

In [39]:
def get_simulants_for_record_ids(record_ids, ground_truth=source_record_ground_truth):
    return tuple(ground_truth.loc[record_id] for record_id in record_ids)

def get_simulants_of_source_records(df, filter_record_ids=None):
    source_record_ids = df.set_index('record_id').source_record_ids
    if filter_record_ids is not None:
        source_record_ids = source_record_ids.apply(lambda r_ids: [r_id for r_id in r_ids if filter_record_ids(r_id)])
    return source_record_ids.apply(get_simulants_for_record_ids).rename('simulant_ids')

# Working with a tuple column is a bit of a pain -- these helpers are for
# operations on this column.
# Oddly, transforming each tuple into a pandas Series (e.g. .apply(pd.Series))
# and using the pandas equivalents of these functions seems to be orders of magnitude slower.

from statistics import multimode

def nunique(data_tuple):
    return len(set(data_tuple))

def mode(data_tuple):
    if len(data_tuple) == 0:
        return None
    return multimode(data_tuple)[0]

### Census Numident

In [40]:
census_numident_simulants = get_simulants_of_source_records(census_numident)
census_numident_simulants

record_id
census_numident_0        (0_15796,)
census_numident_1        (0_14159,)
census_numident_2        (0_15183,)
census_numident_3         (0_7921,)
census_numident_4         (0_3197,)
                            ...    
census_numident_18638    (0_21426,)
census_numident_18639     (0_8798,)
census_numident_18640    (0_19008,)
census_numident_18641    (0_22994,)
census_numident_18642    (0_19793,)
Name: simulant_ids, Length: 18643, dtype: object

In [41]:
census_numident_simulants.apply(nunique).describe()

count    18643.0
mean         1.0
std          0.0
min          1.0
25%          1.0
50%          1.0
75%          1.0
max          1.0
Name: simulant_ids, dtype: float64

In [42]:
census_numident_ground_truth = census_numident_simulants.apply(mode).rename('simulant_id')
census_numident_ground_truth

record_id
census_numident_0        0_15796
census_numident_1        0_14159
census_numident_2        0_15183
census_numident_3         0_7921
census_numident_4         0_3197
                          ...   
census_numident_18638    0_21426
census_numident_18639     0_8798
census_numident_18640    0_19008
census_numident_18641    0_22994
census_numident_18642    0_19793
Name: simulant_id, Length: 18643, dtype: object

### Alternate Name Numident

In [43]:
source_record_simulants = get_simulants_of_source_records(alternate_name_numident)

In [44]:
source_record_simulants.apply(nunique).describe()

count    19067.0
mean         1.0
std          0.0
min          1.0
25%          1.0
50%          1.0
75%          1.0
max          1.0
Name: simulant_ids, dtype: float64

In [45]:
# We take the most common ground truth value.
# Again, as shown above, there are no SSN collisions.
alternate_name_numident_ground_truth = source_record_simulants.apply(mode).rename('simulant_id')
alternate_name_numident_ground_truth

record_id
alternate_name_numident_0        0_11023
alternate_name_numident_1         0_3108
alternate_name_numident_2         0_3922
alternate_name_numident_3        0_17690
alternate_name_numident_4        0_22503
                                  ...   
alternate_name_numident_19062    0_19784
alternate_name_numident_19063    0_19913
alternate_name_numident_19064    0_16231
alternate_name_numident_19065     0_2266
alternate_name_numident_19066    0_17545
Name: simulant_id, Length: 19067, dtype: object

### Alternate DOB Numident

In [46]:
source_record_simulants = get_simulants_of_source_records(alternate_dob_numident)

In [47]:
source_record_simulants.apply(nunique).describe()

count    18887.0
mean         1.0
std          0.0
min          1.0
25%          1.0
50%          1.0
75%          1.0
max          1.0
Name: simulant_ids, dtype: float64

In [48]:
alternate_dob_numident_ground_truth = source_record_simulants.apply(mode).rename('simulant_id')
alternate_dob_numident_ground_truth

record_id
alternate_dob_numident_0        0_11023
alternate_dob_numident_1         0_3108
alternate_dob_numident_2         0_3922
alternate_dob_numident_3        0_17690
alternate_dob_numident_4        0_22503
                                 ...   
alternate_dob_numident_18882    0_19784
alternate_dob_numident_18883    0_19913
alternate_dob_numident_18884    0_16231
alternate_dob_numident_18885     0_2266
alternate_dob_numident_18886    0_17545
Name: simulant_id, Length: 18887, dtype: object

### Name/DOB Reference File

In [49]:
source_record_simulants = get_simulants_of_source_records(name_dob_reference_file)

In [50]:
source_record_simulants.apply(nunique).describe()

count    20014.0
mean         1.0
std          0.0
min          1.0
25%          1.0
50%          1.0
75%          1.0
max          1.0
Name: simulant_ids, dtype: float64

In [51]:
name_dob_reference_file_ground_truth = source_record_simulants.apply(mode).rename('simulant_id')
name_dob_reference_file_ground_truth

record_id
name_dob_reference_file_0        0_11023
name_dob_reference_file_1         0_3108
name_dob_reference_file_2         0_3922
name_dob_reference_file_3        0_17690
name_dob_reference_file_4        0_22503
                                  ...   
name_dob_reference_file_20009    0_10234
name_dob_reference_file_20010     0_7327
name_dob_reference_file_20011     0_8607
name_dob_reference_file_20012    0_13082
name_dob_reference_file_20013    0_17739
Name: simulant_id, Length: 20014, dtype: object

### GeoBase Reference File

In [52]:
source_record_simulants = get_simulants_of_source_records(geobase_reference_file)

In [53]:
# Now there are some collisions, due to "borrowed SSN" noise
source_record_simulants.apply(nunique).describe()

count    32187.000000
mean         1.039892
std          0.201651
min          1.000000
25%          1.000000
50%          1.000000
75%          1.000000
max          3.000000
Name: simulant_ids, dtype: float64

In [54]:
# The most collisions on one SSN
most_collisions = geobase_reference_file.set_index('record_id').loc[source_record_simulants.apply(nunique).sort_values().tail(1).index].reset_index().iloc[0]
most_collisions

record_id                                             geobase_reference_file_20842
ssn                                                                    604-51-9248
first_name                                                                Samantha
middle_name                                                                 Sylvia
last_name                                                                    Aston
date_of_birth                                                             19851102
middle_initial                                                                 NaN
mailing_address_street_number                                                  745
mailing_address_street_name                                        PAULDING AVENUE
mailing_address_unit_number                                                    NaN
mailing_address_po_box                                                         NaN
mailing_address_city                                                       ANYTOWN
mail

In [55]:
# Individual tax filings causing those collisions
taxes[taxes.record_id.isin(most_collisions.source_record_ids)]

,record_id,household_id,first_name,middle_initial,last_name,ssn,mailing_address_street_number,mailing_address_street_name,mailing_address_unit_number,mailing_address_po_box,...,source_record_ids,employer_id,wages,employer_name,employer_street_number,employer_street_name,employer_unit_number,employer_city,employer_state,employer_zipcode
13972,1040_13972,0_2588,Samantha,S,Aston,604-51-9248,745,paulding avenue,NaN,NaN,...,"(1040_13972,)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13975,1040_13975,0_2588,Kira,M,Asto,604-51-9248,745,paulding avenue,NaN,NaN,...,"(1040_13975,)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17463,1040_17463,0_2588,Samantha,S,Aston,604-51-9248,745,paulding avenue,NaN,NaN,...,"(1040_17463,)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25367,1040_25367,0_2588,Samantha,S,Asto,604-51-9248,745,paulding avenue,NaN,NaN,...,"(1040_25367,)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29767,1040_29767,0_2588,Benjamin,W,Aston,604-51-9248,745,paulding avenue,NaN,NaN,...,"(1040_29767,)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33425,1040_33425,0_2588,Samantha,S,Asto,604-51-9248,745,paulding avenue,NaN,NaN,...,"(1040_33425,)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57783,w2_1099_18664,0_2588,Samantha,S,Asto,604-51-9248,745,paulding avenue,NaN,NaN,...,"(w2_1099_18664,)",23,13722,Greenway Med Spa Services,631,regent ave,NaN,Anytown,WA,00000
57784,w2_1099_18665,0_2588,Samantha,S,Aston,604-51-9248,745,paulding avenue,NaN,NaN,...,"(w2_1099_18665,)",39,6019,Gabriel's Liquor & Acting,5153,seville lane,NaN,Anytown,WA,NaN
68564,w2_1099_29445,0_2588,Samantha,S,Aston,604-51-9248,745,paulding avenue,NaN,NaN,...,"(w2_1099_29445,)",23,19821,Greenway Med Spa Services,631,regent ave,NaN,Anytown,WA,00000


In [56]:
# Correct value: who actually has the SSN
ssa_numident[ssa_numident.ssn == most_collisions.ssn]

,record_id,ssn,first_name,middle_name,last_name,date_of_birth,sex,event_type,event_date,source_record_ids
8342,ssa_numident_8342,604-51-9248,Samantha,Sylvia,Aston,19851102,Female,creation,19851102,"(ssa_numident_8342,)"


In [57]:
ssa_numident_ground_truth.loc[ssa_numident[ssa_numident.ssn == most_collisions.ssn].record_id]

record_id
ssa_numident_8342    0_6359
Name: simulant_id, dtype: object

In [58]:
# Let's prioritize the use of SSA records
source_record_simulants_based_on_ssa_only = get_simulants_of_source_records(geobase_reference_file, filter_record_ids=lambda r_id: 'ssa_numident_' in r_id)

In [59]:
geobase_reference_file_ground_truth = (
    source_record_simulants_based_on_ssa_only.apply(mode)
        # When there are no SSA records (ITIN-based), use the standard mode
        .fillna(source_record_simulants.apply(mode))
        .rename('simulant_id')
)
geobase_reference_file_ground_truth

record_id
geobase_reference_file_0        0_11023
geobase_reference_file_1         0_3108
geobase_reference_file_2         0_3922
geobase_reference_file_3         0_3922
geobase_reference_file_4        0_17690
                                 ...   
geobase_reference_file_32182    0_10234
geobase_reference_file_32183     0_7327
geobase_reference_file_32184     0_8607
geobase_reference_file_32185    0_13082
geobase_reference_file_32186    0_17739
Name: simulant_id, Length: 32187, dtype: object

In [60]:
geobase_reference_file_ground_truth.loc[most_collisions.record_id]

'0_6359'

### Get ground truth by SSN

In [61]:
all_ssn_simulant_pairs = pd.concat([
    census_numident.set_index("record_id")[["ssn"]].join(census_numident_ground_truth),
    alternate_name_numident.set_index("record_id")[["ssn"]].join(alternate_name_numident_ground_truth),
    alternate_dob_numident.set_index("record_id")[["ssn"]].join(alternate_dob_numident_ground_truth),
    name_dob_reference_file.set_index("record_id")[["ssn"]].join(name_dob_reference_file_ground_truth),
    geobase_reference_file.set_index("record_id")[["ssn"]].join(geobase_reference_file_ground_truth),
])
all_ssn_simulant_pairs

,ssn,simulant_id
record_id,,
census_numident_0,565-46-1886,0_15796
census_numident_1,221-20-4546,0_14159
census_numident_2,116-26-7703,0_15183
census_numident_3,594-13-8945,0_7921
census_numident_4,702-03-1779,0_3197
...,...,...
geobase_reference_file_32182,999-92-3977,0_10234
geobase_reference_file_32183,999-99-9858,0_7327
geobase_reference_file_32184,9s6-77-5983,0_8607


In [62]:
# The reference file records with a given SSN all have the same (primary) simulant ID
# contributing to them
assert (all_ssn_simulant_pairs.groupby('ssn').simulant_id.nunique() == 1).all()

In [63]:
ssn_to_simulant = all_ssn_simulant_pairs.groupby('ssn').simulant_id.first()
ssn_to_simulant

ssn
001-02-4588    0_11023
001-11-6330     0_3108
001-16-0077     0_3922
001-17-9511    0_17690
001-25-8258    0_22503
                ...   
999-92-3977    0_10234
999-99-9858     0_7327
9s6-77-5983     0_8607
9zs-So-7561    0_13082
9|s-51-3204    0_17739
Name: simulant_id, Length: 19152, dtype: object

## Save results

In [64]:
files = {
    'census_2030': (census_2030, census_2030_ground_truth),
    'census_numident': (census_numident, census_numident_ground_truth),
    'alternate_name_numident': (alternate_name_numident, alternate_name_numident_ground_truth),
    'alternate_dob_numident': (alternate_dob_numident, alternate_dob_numident_ground_truth),
    'geobase_reference_file': (geobase_reference_file, geobase_reference_file_ground_truth),
    'name_dob_reference_file': (name_dob_reference_file, name_dob_reference_file_ground_truth),
}

In [65]:
reference_files = [census_numident, alternate_name_numident, alternate_dob_numident, geobase_reference_file, name_dob_reference_file]
# TODO: Rename the ssn column to explicitly include itins, since this is confusing
all_ssns_itins_in_reference_files = pd.concat([df[["ssn"]] for df in reference_files], ignore_index=True)
ssn_to_pik = (
    all_ssns_itins_in_reference_files.drop_duplicates()
        .reset_index().rename(columns={'index': 'pik'})
        .set_index('ssn').pik
)
ssn_to_pik

ssn
565-46-1886        0
221-20-4546        1
116-26-7703        2
594-13-8945        3
702-03-1779        4
               ...  
999-92-3977    88779
999-99-9858    88780
9s6-77-5983    88781
9zs-So-7561    88782
9|s-51-3204    88783
Name: pik, Length: 19152, dtype: int64

In [66]:
pik_to_simulant = (
    ssn_to_simulant.reset_index()
        .assign(pik=lambda df: df.ssn.map(ssn_to_pik))
        .set_index("pik")
        .simulant_id
)
pik_to_simulant

pik
12710    0_11023
6037      0_3108
4961      0_3922
4367     0_17690
17377    0_22503
          ...   
88779    0_10234
88780     0_7327
88781     0_8607
88782    0_13082
88783    0_17739
Name: simulant_id, Length: 19152, dtype: object

In [67]:
for file_name, (file, ground_truth) in files.items():
    # Add a unique record ID -- could do this within the pipeline, but then it's harder to match up the ground truth
    assert file.record_id.is_unique and ground_truth.index.is_unique
    assert set(file.record_id) == set(ground_truth.index)

    # This tuple column is a pain to serialize
    file = file.drop(columns=['source_record_ids'])

    if file_name != 'census_2030':
        file['pik'] = file.ssn.map(ssn_to_pik)
        assert file.pik.notnull().all()

    ground_truth = ground_truth.reset_index()

    file.to_parquet(f'output/{file_name}_sample.parquet')
    ground_truth.to_parquet(f'output/{file_name}_ground_truth_sample.parquet')

In [68]:
pik_to_simulant.reset_index().to_parquet(f'output/pik_to_simulant_ground_truth.parquet')

In [69]:
# Convert this notebook to a Python script
! cd .. && ./convert_notebook.sh generate_simulated_data/generate_simulated_data_small_sample

[NbConvertApp] Converting notebook generate_simulated_data/generate_simulated_data_small_sample.ipynb to python
